## Direct Implementation

In [2]:
# ==============================================================================
# Baselines – Thesis Evaluation Pipeline (one run per baseline model)
#
# Goal:
# - One model name per baseline (MODEL_NAME)
# - Separate Stage A / Stage B output folders
# - Same log/output structure as other models
#   - outputs/stageA/<MODEL_NAME>/...
#   - outputs/stageB/<MODEL_NAME>/monthly/preds.csv
# ==============================================================================

import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from statsmodels.tsa.ar_model import AutoReg

# --- 1) Path setup -----------------------------------------------------------

def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(6):
        if (cur / "src").exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()


NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT =", PROJECT_ROOT)

# --- 2) Project imports ------------------------------------------------------

from src.config import GlobalConfig, outputs_for_model
from src.tuning import run_stageA, run_stageB
from src.io_timesplits import load_target, load_ifo_features

# --- 3) Baseline model wrapper ----------------------------------------------

class ForecastModel:
    """
    Wrapper for simple time-series baselines, compatible with the tuning pipeline.

    Supported strategies (params['strategy']):
    - 'RW'   : Random walk on ΔIP -> forecast = 0
    - 'Mean' : Expanding grand mean of y
    - 'MA60' : Rolling mean over the last 60 observations (fallback: grand mean)
    - 'AR1'  : AR(1) with intercept via statsmodels.AutoReg

    API:
    - fit(X, y, sample_weight=None)
    - predict(X)
    - predict_one(x_row)
    - get_name()
    - get_feature_importances()
    """

    def __init__(self, params=None):
        self.params = params or {}
        self.strategy = self.params.get("strategy", "RW")
        self._backend_name = f"baseline_{self.strategy}"

        # State
        self.y_train_ = None
        self.prediction_ = 0.0  # Scalar next-step forecast

    def fit(self, X, y, sample_weight=None):
        """Baselines use only the target history y. X and sample_weight are ignored."""
        y_arr = np.asarray(y, dtype=float).ravel()
        self.y_train_ = y_arr

        if self.strategy == "RW":
            self.prediction_ = 0.0

        elif self.strategy == "Mean":
            self.prediction_ = float(np.mean(y_arr))

        elif self.strategy == "MA60":
            if len(y_arr) >= 60:
                self.prediction_ = float(np.mean(y_arr[-60:]))
            else:
                self.prediction_ = float(np.mean(y_arr))

        elif self.strategy == "AR1":
            try:
                model = AutoReg(y_arr, lags=1, trend="c", old_names=False).fit()
                self.prediction_ = float(model.forecast(steps=1)[0])
            except Exception:
                self.prediction_ = float(np.mean(y_arr))

        else:
            raise ValueError(f"Unknown baseline strategy: {self.strategy}")

        return self

    def predict(self, X):
        """Return the same scalar forecast for each row in X."""
        if hasattr(X, "shape"):
            n = X.shape[0]
        else:
            n = len(X)
        return np.full(n, self.prediction_, dtype=float)

    def predict_one(self, x_row):
        """Single-row convenience wrapper."""
        return float(self.prediction_)

    def get_name(self):
        return self._backend_name

    def get_feature_importances(self):
        """Baselines do not have feature importances."""
        return {}

# --- 4) Load data & config ---------------------------------------------------

# Load once and align
y = load_target()
X_ifo = load_ifo_features()  # Used as a dummy design matrix for the pipeline

idx_common = y.index.intersection(X_ifo.index)
y = y.loc[idx_common]
X_ifo = X_ifo.loc[idx_common]

print(f"Data loaded. T = {len(y)} observations.")

def get_thesis_cfg() -> GlobalConfig:
    """Load the standard thesis configuration (splits, policy, etc.)."""
    return GlobalConfig(preset="thesis")

cfg_obj = get_thesis_cfg()

# --- 5) Run each baseline separately ----------------------------------------

strategies = ["RW", "Mean", "MA60", "AR1"]

for strat in strategies:
    MODEL_NAME = f"baseline_{strat.lower()}"
    print("\n" + "=" * 80)
    print(f"Running baseline: {strat}  (MODEL_NAME = '{MODEL_NAME}')")
    print("=" * 80)

    # Initialize output folders for this model
    outputs_for_model(MODEL_NAME)

    # Grid with exactly one configuration
    model_grid = [{"strategy": strat, "seed": 42}]

    # Stage A: keep the single config (for logs/structure consistency)
    shortlist = run_stageA(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        model_grid=model_grid,
        X=X_ifo,
        y=y,
        cfg=cfg_obj,
        keep_top_k_final=1,
        min_survivors_per_block=1,
    )

    # Stage B: final evaluation
    run_stageB(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        shortlist=shortlist,
        X=X_ifo,
        y=y,
        cfg=cfg_obj,
    )

    print(f"Done: {strat}.")
    print(f"Stage A outputs: outputs/stageA/{MODEL_NAME}/")
    print(f"Stage B outputs: outputs/stageB/{MODEL_NAME}/monthly/preds.csv")

print("\nAll baselines completed.")


PROJECT_ROOT = /Users/jonasschernich/Documents/Masterarbeit/Code
INFO in load_ifo_features: Renaming columns to ensure validity.
Daten geladen. T = 407 Beobachtungen.

Starte Baseline: RW  (MODEL_NAME = 'baseline_rw')
[Stage A] Using FULL FE (Gleis 1 & 2) pipeline.
[Stage A][Block 1] train_end=180, OOS=181-200 | configs=1
  - Config 1/1
    · Month 5/20 processed | running...RMSE=1.6761
    · Month 10/20 processed | running...RMSE=1.3459
    · Month 15/20 processed | running...RMSE=1.2615
    · Month 20/20 processed | running...RMSE=1.1305
[Stage A][Block 1] kept 1 configs (floor=1).
[Stage A][Block 2] train_end=200, OOS=201-220 | configs=1
  - Config 1/1
    · Month 5/20 processed | running...RMSE=0.9028
    · Month 10/20 processed | running...RMSE=1.2099
    · Month 15/20 processed | running...RMSE=2.5466
    · Month 20/20 processed | running...RMSE=2.5601
[Stage A][Block 2] kept 1 configs (floor=1).
[Stage A][Block 3] train_end=220, OOS=221-240 | configs=1
  - Config 1/1
    · Month